In [89]:
from utils import generate_data
from layers import NLL
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense,LSTM,GRU,Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nn import reg_nn_lmm

In [69]:
params = {
    "n_fixed_effects": 10,
    "fixed_intercept": 1,
    "X_non_linear": True,
    "Z_non_linear": False,
    "Z_embed_dim_pct": 10,
    "n_per_cat": 3,
}

# qs=[100,1000,10000] # use this if you want 3 z variables
# sig2bs=[0.1,1,10]
X_train, X_test, y_train, y_test, x_cols, dist_matrix, time2measure_dict = generate_data(mode='glmm', qs=[100], sig2e=1, sig2bs=[0.1], sig2bs_spatial=[], q_spatial=[], N=100000, rhos=None, p_censor=None, params=params)#"./conf_files/conf_glmm.conf_glmm.yaml")


In [72]:
df = pd.concat([X_train, X_test], ignore_index=False)
df

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,z0
81168,-0.531767,-0.531129,-0.318579,-0.524463,-0.704684,-0.711323,0.280344,0.265037,0.033684,-0.029767,83
47096,0.057037,-0.372021,0.189338,0.506027,-0.176856,-0.828299,-0.774709,-0.734041,-0.013954,0.444043,49
71956,-0.900382,0.292192,0.742965,-0.558140,-0.265279,0.115898,-0.778816,0.657659,0.206004,-0.447815,73
84795,0.338687,0.148958,0.157310,0.775295,0.333494,-0.792170,-0.576099,0.095555,-0.685807,0.941458,87
23493,0.874637,0.398489,-0.386605,0.719660,0.668492,-0.009923,-0.322420,-0.191939,0.761743,0.201788,23
...,...,...,...,...,...,...,...,...,...,...,...
83490,-0.175773,0.912172,-0.563137,0.529161,0.407686,-0.710036,-0.967755,0.955782,0.083621,-0.051688,85
79596,0.416477,-0.826628,0.091526,-0.169211,0.096359,0.906264,0.499372,0.939270,-0.085170,0.265701,81
34717,0.511446,0.150669,0.909701,-0.652426,0.341461,0.712089,0.119273,0.621494,-0.079138,0.699302,37
37761,0.014751,0.138158,-0.481992,-0.402158,-0.879800,-0.236924,-0.510531,-0.165816,-0.900312,-0.923362,41


In [73]:
target = pd.concat([y_train, y_test], ignore_index=False)
target

81168    1
47096    0
71956    0
84795    1
23493    0
        ..
83490    1
79596    0
34717    0
37761    1
27730    1
Name: y, Length: 100000, dtype: int32

In [74]:
df['label'] = target
df

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,z0,label
81168,-0.531767,-0.531129,-0.318579,-0.524463,-0.704684,-0.711323,0.280344,0.265037,0.033684,-0.029767,83,1
47096,0.057037,-0.372021,0.189338,0.506027,-0.176856,-0.828299,-0.774709,-0.734041,-0.013954,0.444043,49,0
71956,-0.900382,0.292192,0.742965,-0.558140,-0.265279,0.115898,-0.778816,0.657659,0.206004,-0.447815,73,0
84795,0.338687,0.148958,0.157310,0.775295,0.333494,-0.792170,-0.576099,0.095555,-0.685807,0.941458,87,1
23493,0.874637,0.398489,-0.386605,0.719660,0.668492,-0.009923,-0.322420,-0.191939,0.761743,0.201788,23,0
...,...,...,...,...,...,...,...,...,...,...,...,...
83490,-0.175773,0.912172,-0.563137,0.529161,0.407686,-0.710036,-0.967755,0.955782,0.083621,-0.051688,85,1
79596,0.416477,-0.826628,0.091526,-0.169211,0.096359,0.906264,0.499372,0.939270,-0.085170,0.265701,81,0
34717,0.511446,0.150669,0.909701,-0.652426,0.341461,0.712089,0.119273,0.621494,-0.079138,0.699302,37,0
37761,0.014751,0.138158,-0.481992,-0.402158,-0.879800,-0.236924,-0.510531,-0.165816,-0.900312,-0.923362,41,1


In [80]:
#df.to_csv('Synthetic_Data.csv')

In [70]:
X_train

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,z0
81168,-0.531767,-0.531129,-0.318579,-0.524463,-0.704684,-0.711323,0.280344,0.265037,0.033684,-0.029767,83
47096,0.057037,-0.372021,0.189338,0.506027,-0.176856,-0.828299,-0.774709,-0.734041,-0.013954,0.444043,49
71956,-0.900382,0.292192,0.742965,-0.558140,-0.265279,0.115898,-0.778816,0.657659,0.206004,-0.447815,73
84795,0.338687,0.148958,0.157310,0.775295,0.333494,-0.792170,-0.576099,0.095555,-0.685807,0.941458,87
23493,0.874637,0.398489,-0.386605,0.719660,0.668492,-0.009923,-0.322420,-0.191939,0.761743,0.201788,23
...,...,...,...,...,...,...,...,...,...,...,...
78193,-0.339709,-0.933041,0.415370,-0.166107,-0.574307,0.250758,-0.354339,-0.781210,-0.542049,-0.358265,80
40052,-0.691073,-0.592864,-0.403589,0.171708,0.594380,-0.506743,-0.347860,0.772321,0.661877,-0.433515,43
33410,-0.313796,0.401396,-0.469474,-0.724392,0.018785,-0.665830,0.930409,0.229737,0.836275,0.030957,35
10498,0.161213,-0.039404,0.314033,-0.447159,0.578798,-0.092059,-0.468478,0.502907,-0.733544,-0.527874,10


In [71]:
y_train

81168    1
47096    0
71956    0
84795    1
23493    0
        ..
78193    0
40052    1
33410    1
10498    0
26350    1
Name: y, Length: 80000, dtype: int32

In [76]:
x_cols

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']

In [77]:
np.unique(X_train['z0'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [78]:
print(len(y_train))
np.unique(y_train)

80000


array([0, 1])

In [83]:
# simple model using only one z variable
model_simple = Sequential()
model_simple.add(Dense(5, activation='relu'))
model_simple.add(Dense(2, activation='sigmoid'))
model_simple.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model_simple.fit(X_train, y_train, epochs=20, batch_size=256)
# Final evaluation of the model
scores = model_simple.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

313/313 [==============================] - 1s 2ms/step - loss: 0.8103 - accuracy: 0.5729
Epoch 2/20
313/313 [==============================] - 1s 3ms/step - loss: 0.6552 - accuracy: 0.5973
Epoch 3/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6469 - accuracy: 0.5984
Epoch 4/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6416 - accuracy: 0.6097
Epoch 5/20
313/313 [==============================] - 1s 3ms/step - loss: 0.6374 - accuracy: 0.6198
Epoch 6/

In [84]:
# more complex model using 1 z variable
model_simple = Sequential()
model_simple.add(Dense(10, activation='relu'))
model_simple.add(Dense(7, activation='relu'))
model_simple.add(Dense(4, activation='relu'))
model_simple.add(Dense(2, activation='sigmoid'))
model_simple.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model_simple.fit(X_train, y_train, epochs=20, batch_size=256)
# Final evaluation of the model
scores = model_simple.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

313/313 [==============================] - 1s 3ms/step - loss: 0.6880 - accuracy: 0.5245
Epoch 2/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6588 - accuracy: 0.5777
Epoch 3/20
313/313 [==============================] - 0s 2ms/step - loss: 0.6455 - accuracy: 0.5807
Epoch 4/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6387 - accuracy: 0.5860
Epoch 5/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6330 - accuracy: 0.5922
Epoch 6/

In [21]:
# simple model using 3 z variables
model_simple = Sequential()
model_simple.add(Dense(5, activation='relu'))
model_simple.add(Dense(2, activation='sigmoid'))
model_simple.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model_simple.fit(X_train, y_train, epochs=20, batch_size=40)
# Final evaluation of the model
scores = model_simple.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2000/2000 [==============================] - 5s 3ms/step - loss: 1.3688 - accuracy: 0.5435
Epoch 2/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6883 - accuracy: 0.5493
Epoch 3/20
2000/2000 [==============================] - 5s 3ms/step - loss: 0.6883 - accuracy: 0.5493
Epoch 4/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6883 - accuracy: 0.5493
Epoch 5/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6883 - accuracy: 0.549

In [24]:
# more complex model using 3 z variables
model_simple = Sequential()
model_simple.add(Dense(10, activation='relu'))
model_simple.add(Dense(7, activation='relu'))
model_simple.add(Dense(4, activation='relu'))
model_simple.add(Dense(2, activation='sigmoid'))
model_simple.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model_simple.fit(X_train, y_train, epochs=20, batch_size=256)
# Final evaluation of the model
scores = model_simple.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2500/2500 [==============================] - 9s 4ms/step - loss: 0.6931 - accuracy: 0.5492
Epoch 2/20
2500/2500 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5493
Epoch 3/20
2500/2500 [==============================] - 6s 2ms/step - loss: 0.6889 - accuracy: 0.5494
Epoch 4/20
2500/2500 [==============================] - ETA: 0s - loss: 0.7013 - accuracy: 0.54 - 7s 3ms/step - loss: 0.7012 - accuracy: 0.5447
Epoch 5/20
2500/2500 [==============================] - 8s

In [85]:
# Random Forest using 1 z variable

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6599

In [25]:
# Random Forest using 3 z variables

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.57825

In [37]:
col_set = X_train.columns
col_set

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'z0'], dtype='object')

In [38]:
X_train[col_set].shape[1]

11

In [22]:
# LMMNN using 3 z variables

mode = 'glmm'
NUM_ITERATIONS = 1000
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DECAY_RATE = 0.0001

X_input = Input(shape=(X_train[col_set].shape[1],))
y_true_input = Input(shape=(1,))
Z_input = Input(shape=(1,), dtype=tf.int64)
hidden1 = Dense(units=10, activation='relu', input_dim=len(X_train[col_set].values[0]))(X_input)
hidden2 = Dense(units=5, activation='relu')(hidden1)
y_pred_output = Dense(units=1, activation='sigmoid')(hidden2)
nll = NLL(mode, 1.0, [1.0])(y_true_input, y_pred_output, [Z_input])
model = Model(inputs=[X_input, y_true_input, Z_input], outputs=nll)

adam = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=DECAY_RATE, amsgrad=False)
model.compile(optimizer = adam)
# X_train.reset_index(inplace=True)
# X_train.sort_values(by=['z0'], inplace=True)
# y_train = y_train[X_train.index]

history = model.fit([X_train[col_set], y_train, X_train['z0']], None, batch_size=BATCH_SIZE, epochs=10,
                    validation_split=0.1, shuffle=True)

Epoch 1/10
282/282 [==============================] - 27s 96ms/step - loss: 199.9527 - val_loss: 194.5889
Epoch 2/10
282/282 [==============================] - 26s 93ms/step - loss: 199.5365 - val_loss: 194.1446
Epoch 3/10
282/282 [==============================] - 32s 113ms/step - loss: 198.8748 - val_loss: 193.9037
Epoch 4/10
282/282 [==============================] - 25s 87ms/step - loss: 198.6808 - val_loss: 193.7709
Epoch 5/10
282/282 [==============================] - 27s 95ms/step - loss: 195.0841 - val_loss: 180.9470
Epoch 6/10
282/282 [==============================] - 27s 97ms/step - loss: 184.3875 - val_loss: 179.0182
Epoch 7/10
282/282 [==============================] - 26s 93ms/step - loss: 182.6863 - val_loss: 177.7023
Epoch 8/10
282/282 [==============================] - 27s 96ms/step - loss: 181.1982 - val_loss: 176.6341
Epoch 9/10
282/282 [==============================] - 26s 94ms/step - loss: 180.5498 - val_loss: 175.7048
Epoch 10/10
282/282 [========================

In [11]:
model.evaluate([X_test[col_set], y_test, X_test['z0']])

625/625 [==============================] - 8s 14ms/step - loss: 22.1788


22.178842544555664

In [39]:
# LMMNN using 1 z variable

mode = 'glmm'
NUM_ITERATIONS = 1000
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DECAY_RATE = 0.0001

X_input = Input(shape=(X_train[col_set].shape[1],))
y_true_input = Input(shape=(1,))
Z_input = Input(shape=(1,), dtype=tf.int64)
hidden1 = Dense(units=10, activation='relu', input_dim=len(X_train[col_set].values[0]))(X_input)
hidden2 = Dense(units=5, activation='relu')(hidden1)
y_pred_output = Dense(units=1, activation='sigmoid')(hidden2)
nll = NLL(mode, 1.0, [1.0])(y_true_input, y_pred_output, [Z_input])
model = Model(inputs=[X_input, y_true_input, Z_input], outputs=nll)

adam = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=DECAY_RATE, amsgrad=False)
model.compile(optimizer = adam)

# X_train.reset_index(inplace=True)
# X_train.sort_values(by=['z0'], inplace=True)
# y_train = y_train[X_train.index]

history = model.fit([X_train[col_set], y_train, X_train['z0']], None, batch_size=BATCH_SIZE, epochs=10,
                    validation_split=0.1, shuffle=True)

# model.fit([X_train[col_set], y_train, X_train['z0']], None, epochs=20, batch_size=BATCH_SIZE)
# # Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
282/282 [==============================] - 36s 129ms/step - loss: 180.8340 - val_loss: 175.4104
Epoch 2/10
282/282 [==============================] - 36s 127ms/step - loss: 178.2587 - val_loss: 174.1581
Epoch 3/10
282/282 [==============================] - 28s 100ms/step - loss: 177.1214 - val_loss: 173.4776
Epoch 4/10
282/282 [==============================] - 40s 141ms/step - loss: 176.8749 - val_loss: 173.1710
Epoch 5/10
282/282 [==============================] - 40s 142ms/step - loss: 176.6951 - val_loss: 173.0549
Epoch 6/10
282/282 [==============================] - 37s 132ms/step - loss: 175.8944 - val_loss: 170.5000
Epoch 7/10
282/282 [==============================] - 34s 122ms/step - loss: 173.5064 - val_loss: 169.7025
Epoch 8/10
282/282 [==============================] - 37s 132ms/step - loss: 173.2585 - val_loss: 169.5752
Epoch 9/10
282/282 [==============================] - 33s 119ms/step - loss: 173.2996 - val_loss: 169.5060
Epoch 10/10
282/282 [================

In [41]:
scores = model.evaluate([X_test[col_set], y_test, X_test['z0']])
print(scores)
#print("Accuracy: %.2f%%" % (scores[1]*100))

625/625 [==============================] - 3s 4ms/step - loss: 21.7560
21.756011962890625


Create Model using function reg_nn_lmm from nn.py

In [87]:
x_cols = [col for col in X_train.columns if col not in ['z0','z1','z2']]
batch = 100
epochs = 10
patience = 10
qs=[100]#,1000,10000]
q_spatial=[]
n_neurons = [10, 5, 2]
dropout = None
activation = 'relu'
mode = 'glmm'
n_sig2bs = 1
n_sig2bs_spatial = 0
est_cors = []
dist_matrix = None
time2measure_dict = None
spatial_embed_neurons = None
verbose = True
Z_non_linear = False
log_params = False
idx = None
Z_embed_dim_pct = 10


In [88]:
# simple
n_neurons = [5]
y_pred, sigmas, rhos, weibull, n_epochs = reg_nn_lmm(
            X_train, X_test, y_train, y_test, qs, q_spatial, x_cols, batch, epochs, patience,
            n_neurons, dropout, activation, mode,
            n_sig2bs, n_sig2bs_spatial, est_cors, dist_matrix, spatial_embed_neurons, verbose, Z_non_linear, Z_embed_dim_pct, log_params, idx)

Epoch 1/10
720/720 [==============================] - 4s 6ms/step - loss: 67.0956 - val_loss: 65.0391
Epoch 2/10
720/720 [==============================] - 3s 4ms/step - loss: 64.4057 - val_loss: 63.5360
Epoch 3/10
720/720 [==============================] - 4s 5ms/step - loss: 63.0351 - val_loss: 62.5357
Epoch 4/10
720/720 [==============================] - 4s 5ms/step - loss: 62.3010 - val_loss: 62.1152
Epoch 5/10
720/720 [==============================] - 4s 5ms/step - loss: 61.9573 - val_loss: 61.9300
Epoch 6/10
720/720 [==============================] - 4s 6ms/step - loss: 61.7883 - val_loss: 61.8486
Epoch 7/10
720/720 [==============================] - 4s 6ms/step - loss: 61.7039 - val_loss: 61.8190
Epoch 8/10
720/720 [==============================] - 5s 7ms/step - loss: 61.6554 - val_loss: 61.7972
Epoch 9/10
720/720 [==============================] - 4s 6ms/step - loss: 61.6223 - val_loss: 61.7755
Epoch 10/10
720/720 [==============================] - 4s 6ms/step - loss: 61.5933

In [90]:
# round predictions to closest integer
y_pred_class = []
for n in y_pred:
    y_pred_class.append(round(n))

accuracy_score(y_test, y_pred_class)

0.65925

In [43]:
# more complex
y_pred, sigmas, rhos, weibull, n_epochs = reg_nn_lmm(
            X_train, X_test, y_train, y_test, qs, q_spatial, x_cols, batch, epochs, patience,
            n_neurons, dropout, activation, mode,
            n_sig2bs, n_sig2bs_spatial, est_cors, dist_matrix, spatial_embed_neurons, verbose, Z_non_linear, Z_embed_dim_pct, log_params, idx)

Epoch 1/10
720/720 [==============================] - 4s 6ms/step - loss: 67.9445 - val_loss: 66.3830
Epoch 2/10
720/720 [==============================] - 4s 5ms/step - loss: 63.2507 - val_loss: 62.3688
Epoch 3/10
720/720 [==============================] - 4s 5ms/step - loss: 61.0069 - val_loss: 60.9193
Epoch 4/10
720/720 [==============================] - 4s 5ms/step - loss: 60.0911 - val_loss: 60.2229
Epoch 5/10
720/720 [==============================] - 4s 5ms/step - loss: 59.4728 - val_loss: 59.5839
Epoch 6/10
720/720 [==============================] - 4s 5ms/step - loss: 58.8078 - val_loss: 58.7403
Epoch 7/10
720/720 [==============================] - 4s 5ms/step - loss: 58.1545 - val_loss: 58.2428
Epoch 8/10
720/720 [==============================] - 4s 5ms/step - loss: 57.8414 - val_loss: 58.0293
Epoch 9/10
720/720 [==============================] - 4s 5ms/step - loss: 57.6636 - val_loss: 57.8732
Epoch 10/10
720/720 [==============================] - 4s 6ms/step - loss: 57.5537

In [44]:
y_pred

array([0.30334791, 0.32082577, 0.2263784 , ..., 0.13376543, 0.52185211,
       0.153196  ])

In [45]:
y_test

91505    1
42340    0
19847    0
22940    1
39723    0
        ..
15236    1
22404    0
13079    1
31917    0
6239     0
Name: y, Length: 20000, dtype: int32

In [46]:
len(y_pred) == len(y_test)

True

In [52]:
# round predictions to closest integer
y_pred_class = []
for n in y_pred:
    y_pred_class.append(round(n))

y_pred_class

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [54]:
accuracy_score(y_test, y_pred_class)

0.70115